In [26]:
import wave
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
from os import listdir
from os.path import isfile, join
import librosa
import librosa.display
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D,BatchNormalization, Flatten
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.image  as mpimg
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [18]:
train0_path = '../data/gan/0/gan_generated_engiin_X_train.csv'
train1_path = '../data/gan/1/gan_generated_X_train.csv'
test0_path = '../data/gan/0/gan_generated_engiin_X_test.csv'
test1_path = '../data/gan/1/gan_generated_X_test.csv'

In [24]:
df_train0 = pd.read_csv(train0_path)
df_train0['y'] = 0
df_train1 = pd.read_csv(train1_path)
df_train1['y'] = 1
df_test0 = pd.read_csv(test0_path)
df_test0['y'] = 0
df_test1 = pd.read_csv(test1_path)
df_test1['y'] = 1

In [29]:
combined_df = pd.concat([df_train0, df_train1], ignore_index=True)
train_df = shuffle(combined_df, random_state=42)
combined_df = pd.concat([df_test0, df_test1], ignore_index=True)
test_df = shuffle(combined_df, random_state=42)

In [ ]:
y_train = train_df['y']
x_train = train_df.drop(columns = ['y'])

In [39]:
num_labels = 2
filter_size = 3

def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=filter_size,input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dense(64, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))

    model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    # model.add(Dropout(0.5))

    model.add(GlobalAveragePooling2D())

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dense(num_labels, activation='softmax')) 
        
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = 63  # Shape for CNN
model = create_model(input_shape)

c:\Users\home\anaconda3\envs\dxcup\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Cannot convert '63' to a shape.

In [36]:
num_epochs = 30
num_batch_size = 8
history = model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_split=0.3)


Epoch 1/30


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(8, 63), dtype=float32). Expected shape (None, 15, 157, 1), but input has incompatible shape (8, 63)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(8, 63), dtype=float32)
  • training=True
  • mask=None

In [38]:
x_train

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,...,24,25,26,27,28,29,30,31,32,label
206,-375.99005,118.480576,-20.414503,-1.954139,-24.048023,12.559255,-10.250502,-11.466636,-0.062914,9.698340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,-375.53497,118.591736,-20.677240,-1.987650,-24.195997,18.912935,-11.115066,-12.095480,-2.589219,9.695402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.696117,0.268440,0.278559,8.756587,8.373384,14.083437,12.592300,15.373389,20.875595,16.447632
858,-375.40353,118.543465,-20.658283,-1.982052,-24.190529,13.697533,-11.125987,-12.040660,-3.341849,9.710567,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,-381.11038,118.096450,-20.512941,-1.910100,-24.118433,10.749858,-7.443826,-11.819968,-3.549537,9.655209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,-376.01505,118.542030,-20.502392,-1.948830,-24.144762,22.401407,-11.047319,-12.046876,-3.562387,9.642476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.699765,0.266287,0.275081,8.855160,8.339451,14.095439,12.530910,15.373094,20.889585,16.431921
452,-375.10770,118.543030,-20.229042,-1.979926,-24.136505,10.597741,-10.610054,-11.632885,-0.223923,9.666960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.697037,0.269486,0.280505,8.797415,8.369531,14.052298,12.721064,15.370816,20.848078,16.749083


In [2]:
def preprocessing(audio_file, mode):
    # we want to resample audio to 16 kHz
    sr_new = 16000 # 16kHz sample rate
    x, sr = librosa.load(audio_file, sr=sr_new)

    # padding sound 
    # because duration of sound is dominantly 20 s and all of sample rate is 22050
    # we want to pad or truncated sound which is below or above 20 s respectively
    max_len = 5 * sr_new  # length of sound array = time x sample rate
    if x.shape[0] < max_len:
      # padding with zero
      pad_width = max_len - x.shape[0]
      x = np.pad(x, (0, pad_width))
    elif x.shape[0] > max_len:
      # truncated
      x = x[:max_len]
    
    if mode == 'mfcc':
      feature = librosa.feature.mfcc(y=x, sr=sr_new)
    
    elif mode == 'log_mel':
      feature = librosa.feature.melspectrogram(y=x, sr=sr_new, n_mels=128, fmax=8000)
      feature = librosa.power_to_db(feature, ref=np.max)
    

    return feature

In [ ]:
orig_labels = []
orig_preprocessed_data = []
folder_path = "data/orig/healthy/audiowav"
# folder_path = "data/breath/0"
for filename in os.listdir(folder_path):
    orig_labels.append(0)
    data = preprocessing(os.path.join(folder_path, filename), mode = 'mfcc')
    orig_preprocessed_data.append(data)
folder_path = "data/data"
# folder_path = "data/breath/1"

for filename in os.listdir(folder_path):
    orig_labels.append(1)
    data = preprocessing(os.path.join(folder_path, filename), mode = 'mfcc')
    orig_preprocessed_data.append(data)
orig_preprocessed_data = np.array(orig_preprocessed_data)
orig_labels = np.array(orig_labels)
encoder = LabelEncoder()
orig_labels = to_categorical(encoder.fit_transform(orig_labels),num_classes=2) 
unique_values, counts = np.unique(orig_labels, return_counts=True)
orig_preprocessed_data = orig_preprocessed_data.reshape((-1, 20, 157, 1))
# Display unique values and their counts
for value, count in zip(unique_values, counts):
    print(f"Value: {value}, Count: {count}")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(orig_preprocessed_data, orig_labels, test_size=0.2)

In [ ]:
num_rows = 20
num_columns = 157
num_channels = 1


num_labels = 2
filter_size = 2

def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=filter_size,input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=32, kernel_size=filter_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))

    model.add(Conv2D(filters=64, kernel_size=filter_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.5))

    model.add(GlobalAveragePooling2D())

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))

    model.add(Dense(num_labels, activation='softmax')) 
        
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

input_shape = (num_rows, num_columns, num_channels)  # Shape for CNN
model = create_model(input_shape)

# Construct model 


In [ ]:
num_epochs = 100
num_batch_size = 64

callbacks = [
    ModelCheckpoint(
        filepath='kaggle_only_{epoch:02d}.keras',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

history = model.fit(preprocessed_data, oh_labels, batch_size=num_batch_size, epochs=num_epochs, callbacks=callbacks, verbose=1, validation_split=0.2)



duration = datetime.now() - start
print("Training completed in time: ", duration)

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


In [ ]:
pred = model.predict(x_test)
pred_labels = pred.argmax(axis=1)  
true_labels = y_test.argmax(axis=1)  # Assuming test_labels is one-hot encoded

# Calculate the metrics
accuracy = accuracy_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
conf_matrix = confusion_matrix(true_labels, pred_labels)

# Display the confusion matrix
ConfusionMatrixDisplay(conf_matrix).plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()